<a href="https://colab.research.google.com/github/ssundar6087/Deep-Learning-Mini-Course/blob/main/Pytorch/DL_Minicourse_Pytorch_Day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# End to End Network
Today, we'll be training and validating our first neural net. Let's call it **Baby Thanos**. Over the course of the next few lessons, let's see if **Baby Thanos** can collect all the infinity stones and become inevitable. 💠

Don't worry if none of this makes sense yet. Just run all the cells in the notebook and observe the outputs. We'll look at each piece in isolation to better understand deep learning. Let's start 😀

# Image Classification Pytorch

In [ ]:
# Imports
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

## Load Dataset - CIFAR-10
The dataset has 10 classes with 50k training images and 10k test images

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

BATCH_SIZE = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Show Some Images

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images[:4, ...]))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

## Define our Model

In [ ]:
class BabyThanos(nn.Module):
  def __init__(self, in_ftrs, n_classes=10):
    super().__init__()
    # define the layers here
    self.in_ftrs = in_ftrs # Our first neural net will flatten the image to a vector
    self.fc1_dim = 256
    self.fc2_dim = 128
    self.fc1 = nn.Linear(self.in_ftrs, self.fc1_dim)
    self.fc2 = nn.Linear(self.fc1_dim, self.fc2_dim)
    self.clf_head = nn.Linear(self.fc2_dim, n_classes)
  
  # define the flow of data through the layers here
  def forward(self, x):
    x = x.reshape(-1, self.in_ftrs) # Flatten the image
    x = self.fc1(x)
    x = F.relu(x) # Pass output through activation layer
    x = self.fc2(x)
    x = F.relu(x) 
    x = self.clf_head(x) # (batch_size, num_classes)
    return x
    

In [ ]:
IN_FTRS = 32 * 32 * 3 # CIFAR-10 images are 32 x 32 and have 3 channels
net = BabyThanos(in_ftrs=IN_FTRS, n_classes=10)

In [ ]:
print(net)

## Define Optimizer & Loss Function
These two functions allow us to help baby thanos learn from his mistakes.

In [ ]:
criterion = nn.CrossEntropyLoss() # Loss Function
optimizer = optim.SGD(net.parameters(), lr=0.001) # Optimizer 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Train and Evaluate the Network
The training and validation loops call the same set of functions over and over, so we'll package them into separate functions. Note that the validation loop does not have any optimizer calls. 

In [ ]:
from tqdm.notebook import tqdm
def train_step(model, train_loader, optimizer, criterion):
  model.train()
  epoch_loss = []
  total, correct = 0, 0

  for i, batch in tqdm(enumerate(train_loader), 
                       total=len(train_loader),
                       leave=False,
                       ):
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad() # Erase history - clean slate

    predictions = model(images) # forward -> model (images) -> make guesses on labels
    loss = criterion(predictions, labels) # how did I do?
    epoch_loss.append(loss.item())
    _, predicted = torch.max(predictions.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item() # Accuracy score
    loss.backward() # backward pass
    optimizer.step() # Update the weights using gradients
  
  return np.mean(epoch_loss), correct / total


In [ ]:
def valid_step(model, val_loader, criterion):
  model.eval()
  epoch_loss = []
  total, correct = 0, 0

  with torch.no_grad():
    for i, batch in tqdm(enumerate(val_loader), 
                        total=len(val_loader),
                        leave=False,
                        ):
      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)
      # Note that there's no optimizer here
      predictions = model(images)
      loss = criterion(predictions, labels)
      epoch_loss.append(loss.item())
      _, predicted = torch.max(predictions.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  
  return np.mean(epoch_loss), correct / total

In [ ]:
EPOCHS = 20
net = net.to(device)
losses = {"train_loss": [], "val_loss": []}
accuracies = {"train_acc": [], "val_acc": []}
epochs = []
for epoch in tqdm(range(EPOCHS), total=EPOCHS):
  train_loss, train_acc = train_step(net, 
                                     trainloader, 
                                     optimizer, 
                                     criterion,)
  
  val_loss, val_acc = valid_step(net, 
                                 testloader, 
                                 criterion,
                                 )
  
  losses["train_loss"].append(train_loss)
  losses["val_loss"].append(val_loss)
  accuracies["train_acc"].append(train_acc)
  accuracies["val_acc"].append(val_acc)
  epochs.append(epoch)

  print(f'[{epoch + 1}] train loss: {train_loss}  train accuracy: {train_acc}  val loss: {val_loss}  val accuracy: {val_acc}')

## Plot the Loss and Accuracy of our Model

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(epochs, losses["train_loss"], label="train")
plt.plot(epochs, losses["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid("on")
plt.legend()
plt.title("Loss vs Epochs")

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(epochs, accuracies["train_acc"], label="train")
plt.plot(epochs, accuracies["val_acc"], label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid("on")
plt.legend()
plt.title("Accuracy vs Epochs")